# BERT Fine-Tuning

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd

# 1. Setup

## 1.1. GPU


In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [ ]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


## 1.2. Installing the Hugging Face Library


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 2. Loading Dataset


## 2.1. Download & Extract

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/rucode_data/train.csv')
test = pd.read_csv('/content/drive/MyDrive/rucode_data/public_test.csv')
print(train.shape, test.shape)
train.head()

(6151, 3) (1636, 2)


,context,answer,label
0,**Вы:** Спасибо большое)).\nКакую музыку слуша...,"Ой, ну тогда давай я тебе порекомендую что-ниб...",ai
1,**Вы:** Я тоже не замужем. Ищу своего принца. ...,"Может быть и тихие, но совсем не мирные, могут...",people
2,**Вы:** Ивановская область.\n\n**Собеседник:**...,В деревне у меня на даче.,ai
3,**Вы:** Особенно чинить их.\n\n**Собеседник:**...,"В доме - полный порядок, все на своем месте.",ai
4,**Вы:** В свободное время.\n\n**Собеседник:** ...,"Да, я фотографирую, это моя страсть и хобби.",ai


In [ ]:
train['label'] = train['label'].replace({'ai':0, 'people':1})

In [ ]:
train['text'] = train['answer'] + ' ' + train['context']
train.head()

,context,answer,label,text
0,**Вы:** Спасибо большое)).\nКакую музыку слуша...,"Ой, ну тогда давай я тебе порекомендую что-ниб...",0,"Ой, ну тогда давай я тебе порекомендую что-ниб..."
1,**Вы:** Я тоже не замужем. Ищу своего принца. ...,"Может быть и тихие, но совсем не мирные, могут...",1,"Может быть и тихие, но совсем не мирные, могут..."
2,**Вы:** Ивановская область.\n\n**Собеседник:**...,В деревне у меня на даче.,0,В деревне у меня на даче. **Вы:** Ивановская о...
3,**Вы:** Особенно чинить их.\n\n**Собеседник:**...,"В доме - полный порядок, все на своем месте.",0,"В доме - полный порядок, все на своем месте. *..."
4,**Вы:** В свободное время.\n\n**Собеседник:** ...,"Да, я фотографирую, это моя страсть и хобби.",0,"Да, я фотографирую, это моя страсть и хобби. *..."


In [ ]:
df = train[['label', 'text']]

In [ ]:
def f(s):
  s = s.replace('\n', '')
  s = s.replace('*', '')
  return s
train['text'] = train['text'].apply(f)
train.head()

,context,answer,label,text
0,**Вы:** Спасибо большое)).\nКакую музыку слуша...,"Ой, ну тогда давай я тебе порекомендую что-ниб...",0,"Ой, ну тогда давай я тебе порекомендую что-ниб..."
1,**Вы:** Я тоже не замужем. Ищу своего принца. ...,"Может быть и тихие, но совсем не мирные, могут...",1,"Может быть и тихие, но совсем не мирные, могут..."
2,**Вы:** Ивановская область.\n\n**Собеседник:**...,В деревне у меня на даче.,0,В деревне у меня на даче. Вы: Ивановская облас...
3,**Вы:** Особенно чинить их.\n\n**Собеседник:**...,"В доме - полный порядок, все на своем месте.",0,"В доме - полный порядок, все на своем месте. В..."
4,**Вы:** В свободное время.\n\n**Собеседник:** ...,"Да, я фотографирую, это моя страсть и хобби.",0,"Да, я фотографирую, это моя страсть и хобби. В..."


## 2.2. Parse



Let's extract the sentences and labels of our training set as numpy ndarrays.

In [ ]:
# Get the lists of sentences and their labels.
sentences = df.text.values
labels = df.label.values

# 3. Tokenization & Input Formatting


## 3.1. BERT Tokenizer

In [ ]:
from transformers import BertTokenizer

print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [ ]:
print(' Original: ', sentences[0])
print('Tokenized: ', tokenizer.tokenize(sentences[0]))
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

 Original:  Ой, ну тогда давай я тебе порекомендую что-нибудь послушать? Я недавно открыл для себя классический рок, например, Queen, Led Zeppelin, Pink Floyd - это просто космос! А еще мне нравится электронная музыка, например, Daft Punk, The Chemical Brothers, Justice. А ты какую музыку любишь? Может, я тоже что-то новое для себя открою? 😊 **Вы:** Спасибо большое)).
Какую музыку слушаешь?

**Собеседник:** русский рэп в основном.
ну там, фейс, элджей знаешь же?
или что ты слушаешь?).

**Вы:** Эм..нет😱.
Tokenized:  ['о', '##и', ',', 'н', '##у', 'т', '##о', '##г', '##д', '##а', 'д', '##а', '##в', '##а', '##и', 'я', 'т', '##е', '##б', '##е', 'п', '##о', '##р', '##е', '##к', '##о', '##м', '##е', '##н', '##д', '##у', '##ю', 'ч', '##т', '##о', '-', 'н', '##и', '##б', '##у', '##д', '##ь', 'п', '##о', '##с', '##л', '##у', '##ш', '##а', '##т', '##ь', '?', 'я', 'н', '##е', '##д', '##а', '##в', '##н', '##о', 'о', '##т', '##к', '##р', '##ы', '##л', 'д', '##л', '##я', 'с', '##е', '##б', '##я', 'к'

## 3.2. Required Formatting

## 3.2.1 Sentences to IDs

In [ ]:
input_ids = []

for sent in sentences:
    encoded_sent = tokenizer.encode(
                        sent,
                        add_special_tokens = True,
                        #max_length = 128,          # Truncate all sentences.
                        #return_tensors = 'pt',     # Return pytorch tensors.
                   )
    input_ids.append(encoded_sent)

print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Token indices sequence length is longer than the specified maximum sequence length for this model (596 > 512). Running this sequence through the model will result in indexing errors


Original:  Ой, ну тогда давай я тебе порекомендую что-нибудь послушать? Я недавно открыл для себя классический рок, например, Queen, Led Zeppelin, Pink Floyd - это просто космос! А еще мне нравится электронная музыка, например, Daft Punk, The Chemical Brothers, Justice. А ты какую музыку любишь? Может, я тоже что-то новое для себя открою? 😊 **Вы:** Спасибо большое)).
Какую музыку слушаешь?

**Собеседник:** русский рэп в основном.
ну там, фейс, элджей знаешь же?
или что ты слушаешь?).

**Вы:** Эм..нет😱.
Token IDs: [101, 1193, 10325, 1010, 1192, 29748, 1197, 14150, 29741, 29742, 10260, 1184, 10260, 25529, 10260, 10325, 1210, 1197, 15290, 29740, 15290, 1194, 14150, 16856, 15290, 23925, 14150, 29745, 15290, 18947, 29742, 29748, 29757, 1202, 22919, 14150, 1011, 1192, 10325, 29740, 29748, 29742, 23742, 1194, 14150, 29747, 29436, 29748, 29753, 10260, 22919, 23742, 1029, 1210, 1192, 15290, 29742, 10260, 25529, 18947, 14150, 1193, 22919, 23925, 16856, 29113, 29436, 1184, 29436, 17432, 1196, 152

## 3.3. Padding & Truncating

Pad and truncate our sequences so that they all have the same length, `MAX_LEN`.

In [ ]:
print('Max sentence length: ', max([len(sen) for sen in input_ids]))

Max sentence length:  1054


Given that, let's choose MAX_LEN = 64 and apply the padding.

In [ ]:
!pip install Keras-Preprocessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from keras_preprocessing.sequence import pad_sequences
MAX_LEN = 256

print('\nPadding/truncating all sentences to %d values...' % MAX_LEN)
print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))

input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long",
                          value=0, truncating="post", padding="post")

print('\nDone.')


Padding/truncating all sentences to 256 values...

Padding token: "[PAD]", ID: 0

Done.


## 3.4. Attention Masks

In [ ]:
# Create attention masks
attention_masks = []

for sent in input_ids:
    att_mask = [int(token_id > 0) for token_id in sent]
    attention_masks.append(att_mask)

## 3.5. Training & Validation Split


In [ ]:
from sklearn.model_selection import train_test_split
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels,
                                                            random_state=13, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels,
                                             random_state=13, test_size=0.1)

## 3.6. Converting to PyTorch Data Types

In [ ]:
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)

train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)

train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


# 4. Train Our Classification Model

## 4.1. BertForSequenceClassification

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False,
)
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))
print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

## 4.2. Optimizer & Learning Rate Scheduler

In [ ]:
optimizer = AdamW(model.parameters(), lr = 2e-5, eps = 1e-8)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from transformers import get_linear_schedule_with_warmup

epochs = 10
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)

## 4.3. Training Loop

In [ ]:
import numpy as np
from sklearn.metrics import f1_score

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def flat_f1(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, pred_flat)

Helper function for formatting elapsed times.


In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))


We're ready to kick off the training!

In [ ]:
import random

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128


seed_val = 13

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

loss_values = []

for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_loss = 0

    # Put the model into training mode. Don't be mislead--the call to
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)

            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader.
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because
        # accumulating the gradients is "convenient while training RNNs".
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()

        # Perform a forward pass (evaluate the model on this training batch).
        # This will return the loss (rather than the model output) because we
        # have provided the `labels`.
        # The documentation for this `model` function is here:
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model(b_input_ids,
                    token_type_ids=None,
                    attention_mask=b_input_mask,
                    labels=b_labels)

        # The call to `model` always returns a tuple, so we need to pull the
        # loss value out of the tuple.
        loss = outputs[0]

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value
        # from the tensor.
        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)

    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))

    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables
    eval_loss, eval_accuracy = 0, 0
    eval_f1 = 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:

        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)

        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch

        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():

            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # token_type_ids is the same as the "segment ids", which
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here:
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = model(b_input_ids,
                            token_type_ids=None,
                            attention_mask=b_input_mask)

        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs[0]

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        tmp_eval_f1 = flat_f1(logits, label_ids)

        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy
        eval_f1 += tmp_eval_f1

        # Track the number of batches
        nb_eval_steps += 1

    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  F1: {0:.2f}".format(eval_f1/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 10 ========
Training...
  Batch    40  of    173.    Elapsed: 0:00:58.
  Batch    80  of    173.    Elapsed: 0:01:53.
  Batch   120  of    173.    Elapsed: 0:02:50.
  Batch   160  of    173.    Elapsed: 0:03:47.

  Average training loss: 0.36
  Training epcoh took: 0:04:05

Running Validation...
  Accuracy: 0.86
  F1: 0.87
  Validation took: 0:00:10

======== Epoch 2 / 10 ========
Training...
  Batch    40  of    173.    Elapsed: 0:00:56.
  Batch    80  of    173.    Elapsed: 0:01:53.
  Batch   120  of    173.    Elapsed: 0:02:49.
  Batch   160  of    173.    Elapsed: 0:03:46.

  Average training loss: 0.32
  Training epcoh took: 0:04:04

Running Validation...
  Accuracy: 0.85
  F1: 0.85
  Validation took: 0:00:10

======== Epoch 3 / 10 ========
Training...


Let's take a look at our training loss over all batches:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

# Use plot styling from seaborn.
sns.set(style='darkgrid')

# Increase the plot size and font size.
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)

# Plot the learning curve.
plt.plot(loss_values, 'b-o')

# Label the plot.
plt.title("Training loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")

plt.show()

# 5. Performance On Test Set